In [5]:
%run Markovify.py

In [6]:
example.modeling_df['hist_ind'].value_counts()

0.0    139700
1.0     45014
Name: hist_ind, dtype: int64

In [7]:
example.modeling_df['nextaction'].value_counts()

 1.0      120894
 3.0       25843
 104.0     13047
 7.0        7844
 4.0        7407
 6.0        7104
-1.0        2575
Name: nextaction, dtype: int64

In [8]:
example.rfc_test()

0.7108830426476771

In [9]:
example.rfc_fit()

In [10]:
example.rfc_score()

0.77665566423401866

In [11]:
example.test_y.value_counts()

 1.0      49391
 3.0       7681
 104.0     3417
 4.0       2742
 7.0       1697
-1.0       1118
Name: nextaction, dtype: int64